In [1]:
import requests
import time
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import re

In [2]:
url = 'https://sthaniya.gov.np/gis/website/webdataquery.php?province=0'
page = requests.get(url, timeout=1000, headers={'User-Agent': 'Mozilla/5.0'})
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
tbl = soup.find('table')

In [4]:
table = pd.read_html(str(tbl))[0]

In [5]:
table

,क्र.स.,स्थानीय तहको नाम,जिल्ला,स्थानीय तहको प्रकार,प्रदेश,वेवसाईट,कैफियत
0,1,फुङलिङ नगरपालिका,ताप्लेजुङ,नगरपालिका,प्रदेश नम्बर १,phunglingmun.gov.np,तयार
1,2,आठराई त्रिवेणी गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,aathraitribenimun.gov.np,तयार
2,3,सिदिङ्वा गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,sidingbamun.gov.np,तयार
3,4,फक्ताङलुङ गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,phaktanglungmun.gov.np,तयार
4,5,मिक्वाखोला गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,mikwakholamun.gov.np,तयार
...,...,...,...,...,...,...,...
748,749,सुनवल नगरपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),नगरपालिका,लुम्बिनी प्रदेश,sunwalmun.gov.np,तयार
749,750,सुस्ता गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,sustamun.gov.np,तयार
750,751,पाल्हीनन्दन गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,palhinandanmun.gov.np,तयार
751,752,प्रतापपुर गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,pratappurmun.gov.np,तयार


In [6]:
whole_data = []
unable_to_scrap = []

In [7]:
table.columns

Index(['क्र.स.', 'स्थानीय तहको नाम', 'जिल्ला', 'स्थानीय तहको प्रकार', 'प्रदेश',
       'वेवसाईट', 'कैफियत'],
      dtype='object')

In [8]:
table_data = np.array(table)

In [9]:
table_data

array([[1, 'फुङलिङ नगरपालिका', 'ताप्लेजुङ', ..., 'प्रदेश नम्बर १',
        'phunglingmun.gov.np', 'तयार'],
       [2, 'आठराई त्रिवेणी गाउँपालिका', 'ताप्लेजुङ', ...,
        'प्रदेश नम्बर १', 'aathraitribenimun.gov.np', 'तयार'],
       [3, 'सिदिङ्वा गाउँपालिका', 'ताप्लेजुङ', ..., 'प्रदेश नम्बर १',
        'sidingbamun.gov.np', 'तयार'],
       ...,
       [751, 'पाल्हीनन्दन गाउँपालिका',
        'नवलपरासी (बर्दघाट सुस्ता पश्चिम)', ..., 'लुम्बिनी प्रदेश',
        'palhinandanmun.gov.np', 'तयार'],
       [752, 'प्रतापपुर गाउँपालिका', 'नवलपरासी (बर्दघाट सुस्ता पश्चिम)',
        ..., 'लुम्बिनी प्रदेश', 'pratappurmun.gov.np', 'तयार'],
       [753, 'सरावल गाउँपालिका', 'नवलपरासी (बर्दघाट सुस्ता पश्चिम)', ...,
        'लुम्बिनी प्रदेश', 'sarawalmun.gov.np', 'तयार']], dtype=object)

In [10]:
def extractEmail(url):
    mun_page = requests.get(url, timeout=5000, headers={'User-Agent': 'Mozilla/5.0'})
    page_soup = BeautifulSoup(mun_page.content, 'html.parser')
    page_content = page_soup.text
    emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', page_content)
    emails = ', '.join(emails)
    return emails

In [11]:
# extractEmail('http://'+"phunglingmun.gov.np")

In [12]:
counter = 1
for td in table_data:
    print(counter)
    mun_url = 'http://'+td[5]
    print('Scrapping', 'http://'+td[5])
    try:
        email = extractEmail(mun_url)
        whole_data.append({
            'sn': td[0],
            'municipality': td[1],
            'District': td[2],
            'Local_level_Type': td[3],
            'Province': td[4],
            'website': mun_url,
            'Email': email,
        })
        counter = counter + 1
        print('Completed\n')
    except:
        unable_to_scrap.append(int(td[0])-1)
        print('unabel to scrap')

1
Scrapping http://phunglingmun.gov.np
Completed

2
Scrapping http://aathraitribenimun.gov.np
Completed

3
Scrapping http://sidingbamun.gov.np
Completed

4
Scrapping http://phaktanglungmun.gov.np
Completed

5
Scrapping http://mikwakholamun.gov.np
Completed

6
Scrapping http://meringdenmun.gov.np
Completed

7
Scrapping http://maiwakholamun.gov.np
Completed

8
Scrapping http://yangwarakmuntaplejung.gov.np
Completed

9
Scrapping http://sirijanghamun.gov.np
Completed

10
Scrapping http://phidimmun.gov.np
Completed

11
Scrapping http://phalelungmun.gov.np
Completed

12
Scrapping http://phalgunandamun.gov.np
Completed

13
Scrapping http://hilihangmun.gov.np
Completed

14
Scrapping http://kummayakmun.gov.np
Completed

15
Scrapping http://miklajungmunpanchthar.gov.np
Completed

16
Scrapping http://tumwewamun.gov.np
Completed

17
Scrapping http://yangwarakmunpanchthar.gov.np
Completed

18
Scrapping http://ilammun.gov.np
Completed

19
Scrapping http://deumaimun.gov.np
Completed

20
Scrapping htt

Completed

160
Scrapping http://mirchaiyamun.gov.np
Completed

161
Scrapping http://kalyanpurmun.gov.np
Completed

162
Scrapping http://karjanhamun.gov.np
Completed

163
Scrapping http://sukhipurmun.gov.np
Completed

164
Scrapping http://bhagwanpurmun.gov.np
Completed

165
Scrapping http://aurahimunsiraha.gov.np
Completed

166
Scrapping http://bishnupurmunsiraha.gov.np
Completed

167
Scrapping http://bariyapattimun.gov.np
Completed

168
Scrapping http://laxmipurpatarimun.gov.np
Completed

169
Scrapping http://narahamun.gov.np
Completed

170
Scrapping http://sakhuwanankarkattimun.gov.np
Completed

171
Scrapping http://anarmamun.gov.np
Completed

172
Scrapping http://nawarajpurmun.gov.np
Completed

173
Scrapping http://janakpurmun.gov.np
Completed

174
Scrapping http://kshireshwornathmun.gov.np
Completed

175
Scrapping http://ganeshmancharnathmun.gov.np
Completed

176
Scrapping http://dhanushadhammun.gov.np
Completed

177
Scrapping http://nagrainmun.gov.np
Completed

178
Scrapping http:/

Completed

314
Scrapping http://dhunibeshimun.gov.np
Completed

315
Scrapping http://neelakanthamun.gov.np
Completed

316
Scrapping http://khaniyabasmun.gov.np
Completed

317
Scrapping http://gajurimun.gov.np
Completed

318
Scrapping http://galchhimun.gov.np
Completed

319
Scrapping http://gangajamunamun.gov.np
Completed

320
Scrapping http://jwalamukhimun.gov.np
Completed

321
Scrapping http://thakremun.gov.np
Completed

322
Scrapping http://netrawatimun.gov.np
Completed

323
Scrapping http://benighatrorangmun.gov.np
Completed

324
Scrapping http://rubivalleymun.gov.np
Completed

325
Scrapping http://siddhalekmun.gov.np
Completed

326
Scrapping http://tripurasundarimundhading.gov.np
Completed

327
Scrapping http://hetaudamun.gov.np
Completed

328
Scrapping http://thahamun.gov.np
Completed

329
Scrapping http://indrasarowarmun.gov.np
Completed

330
Scrapping http://kailashmun.gov.np
Completed

331
Scrapping http://bakaiyamun.gov.np
Completed

332
Scrapping http://bagmatimunmakawanpur.g

Completed

471
Scrapping http://ismamun.gov.np
Completed

472
Scrapping http://kaligandakimungulmi.gov.np
Completed

473
Scrapping http://gulmidarbarmun.gov.np
Completed

474
Scrapping http://satyawatimun.gov.np
Completed

475
Scrapping http://chandrakotmun.gov.np
Completed

476
Scrapping http://rurumun.gov.np
Completed

477
Scrapping http://chhatrakotmun.gov.np
Completed

478
Scrapping http://dhurkotmun.gov.np
Completed

479
Scrapping http://madanemun.gov.np
Completed

480
Scrapping http://malikamungulmi.gov.np
Completed

481
Scrapping http://rampurmun.gov.np
Completed

482
Scrapping http://tansenmun.gov.np
Completed

483
Scrapping http://nisdimun.gov.np
Completed

484
Scrapping http://purbakholamun.gov.np
Completed

485
Scrapping http://rambhamun.gov.np
Completed

486
Scrapping http://mathagadhimun.gov.np
Completed

487
Scrapping http://tinaumun.gov.np
Completed

488
Scrapping http://baganaskalimun.gov.np
Completed

489
Scrapping http://ribdikotmun.gov.np
Completed

490
Scrapping htt

Completed

628
Scrapping http://tatopanimun.gov.np
Completed

629
Scrapping http://patarasimun.gov.np
Completed

630
Scrapping http://khandachakramun.gov.np
Completed

631
Scrapping http://raskotmun.gov.np
Completed

632
Scrapping http://tilagufamun.gov.np
Completed

633
Scrapping http://pachaljharanamun.gov.np
Completed

634
Scrapping http://sannitrivenimun.gov.np
Completed

635
Scrapping http://narharinathmun.gov.np
Completed

636
Scrapping http://kalikamunkalikot.gov.np
Completed

637
Scrapping http://mahawaimun.gov.np
Completed

638
Scrapping http://palatamun.gov.np
Completed

639
Scrapping http://chhayanathraramun.gov.np
Completed

640
Scrapping http://mugumkarmarongmun.gov.np
Completed

641
Scrapping http://sorumun.gov.np
Completed

642
Scrapping http://khatyadmun.gov.np
Completed

643
Scrapping http://simkotmun.gov.np
Completed

644
Scrapping http://namkhamun.gov.np
Completed

645
Scrapping http://kharpunathmun.gov.np
Completed

646
Scrapping http://sarkegadmun.gov.np
Completed


In [13]:
whole_data

[{'sn': 1,
  'municipality': 'फुङलिङ नगरपालिका',
  'District': 'ताप्लेजुङ',
  'Local_level_Type': 'नगरपालिका',
  'Province': 'प्रदेश नम्बर १',
  'website': 'http://phunglingmun.gov.np',
  'Email': 'info@taplejungmun.gov.np'},
 {'sn': 2,
  'municipality': 'आठराई त्रिवेणी गाउँपालिका',
  'District': 'ताप्लेजुङ',
  'Local_level_Type': 'गाउँपालिका',
  'Province': 'प्रदेश नम्बर १',
  'website': 'http://aathraitribenimun.gov.np',
  'Email': 'aathrai.tribenimun@gmail.com'},
 {'sn': 3,
  'municipality': 'सिदिङ्वा गाउँपालिका',
  'District': 'ताप्लेजुङ',
  'Local_level_Type': 'गाउँपालिका',
  'Province': 'प्रदेश नम्बर १',
  'website': 'http://sidingbamun.gov.np',
  'Email': 'sidingbarmtpj@gmail.com, ito.sidingba@gmail.com, sidingbarmtpj@gmail.com'},
 {'sn': 4,
  'municipality': 'फक्ताङलुङ गाउँपालिका',
  'District': 'ताप्लेजुङ',
  'Local_level_Type': 'गाउँपालिका',
  'Province': 'प्रदेश नम्बर १',
  'website': 'http://phaktanglungmun.gov.np',
  'Email': 'phaktanglungmun@gmail.com'},
 {'sn': 5,
  'mun

In [14]:
email_df = pd.DataFrame(whole_data)

In [15]:
email_df.to_excel("municipality.xlsx")

In [16]:
unable_to_scrap

[195, 746]

In [17]:
email_df

,sn,municipality,District,Local_level_Type,Province,website,Email
0,1,फुङलिङ नगरपालिका,ताप्लेजुङ,नगरपालिका,प्रदेश नम्बर १,http://phunglingmun.gov.np,info@taplejungmun.gov.np
1,2,आठराई त्रिवेणी गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,http://aathraitribenimun.gov.np,aathrai.tribenimun@gmail.com
2,3,सिदिङ्वा गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,http://sidingbamun.gov.np,"sidingbarmtpj@gmail.com, ito.sidingba@gmail.co..."
3,4,फक्ताङलुङ गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,http://phaktanglungmun.gov.np,phaktanglungmun@gmail.com
4,5,मिक्वाखोला गाउँपालिका,ताप्लेजुङ,गाउँपालिका,प्रदेश नम्बर १,http://mikwakholamun.gov.np,"ito.mikwakholamun@gmail.com, mikwakhola.rm@gma..."
...,...,...,...,...,...,...,...
746,749,सुनवल नगरपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),नगरपालिका,लुम्बिनी प्रदेश,http://sunwalmun.gov.np,"sunwalmunicipality@gmail.com, suchanaadhikari@..."
747,750,सुस्ता गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,http://sustamun.gov.np,"ito.sustamun@gmail.com, ito.sustamun@gmail.com..."
748,751,पाल्हीनन्दन गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,http://palhinandanmun.gov.np,"ito.palhinandanmun@gmail.com, info@palhinandan..."
749,752,प्रतापपुर गाउँपालिका,नवलपरासी (बर्दघाट सुस्ता पश्चिम),गाउँपालिका,लुम्बिनी प्रदेश,http://pratappurmun.gov.np,"ito.pratappurmun@gmail.com, ito.pratappurmun@g..."
